In [1]:
import pandas as pd
import codecs

##Import section to ensure the main directory is in the path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Importing from the main directory now
from dbf900_main_bytes import parse_record
from dbf900_layouts_bytes import dbf900_layout

In [2]:
##From https://github.com/skylerbast/TxRRC_data
##Generates the block of bytes from the file.
## Should be added to dbf900_main_bytes when it is working
def yield_blocks(file, n):
    block_bytes = file.read(n)
    while block_bytes:
        yield block_bytes
        block_bytes = file.read(n)

In [3]:
##From https://github.com/skylerbast/TxRRC_data
##converts bytes to string through codecs decoding
## Should be added to dbf900_main_bytes when it is working
def display(data):
    ebcdic_decoder = codecs.getdecoder('cp1140')
    decoded = ebcdic_decoder(data)
    val = decoded[0]
    
    return val

In [4]:
file_path = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

file = open(file_path, 'rb') ##Opens the .ebc file and reads it as bytes

split_records = []  ## array for records
n  = 247 ##Number of bytes for each record in dbf900.ebc file

In [5]:
well_ct = 0 ##counter for number of wells
location_ct = 0 ##counter for wells with lat-long location sections

for block in yield_blocks(file, n): ##for each 
    startval = display(block[0:2]) ##
    if startval == '01': ##check for wbroot record
        well_ct+=1 ##increase well count by one
        API = display(block[2:10]) ##api value in records '01'
    
    if startval == '13': ##checking only the pic_signed section
        location_ct+=1  ##increase location count by one
        
        layout = dbf900_layout(startval)['layout'] ##Retrieves layout for the block

        ##reading the records needs to be in bytes and not decode the whole block before parsing
        parsed_vals = parse_record(block, layout)  ##Decodes the block based on layout

        split_records.append([API,parsed_vals])  ##adds record to the array

    if location_ct > 50: ##limiting number of wells used in example
        break

original sent bytes: b'\xf0\xf3\xf2\xf0\xf0\xf1\xf8\xf9\xf4\xc3'
converted to array: array('B', [240, 243, 242, 240, 240, 241, 248, 249, 244, 195])
result by adding array: 320018943.0
last byte: 195 195
final val to be sent back: 32.0018943
original sent bytes: b'\xf0\xf9\xf6\xf0\xf3\xf2\xf3\xf2\xf0\xc2'
converted to array: array('B', [240, 249, 246, 240, 243, 242, 243, 242, 240, 194])
result by adding array: 960323202.0
last byte: 194 194
final val to be sent back: 96.0323202
original sent bytes: b'\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xc0'
converted to array: array('B', [240, 240, 240, 240, 240, 240, 240, 240, 240, 192])
result by adding array: 0.0
last byte: 192 192
final val to be sent back: 0.0
original sent bytes: b'\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xc0'
converted to array: array('B', [240, 240, 240, 240, 240, 240, 240, 240, 240, 192])
result by adding array: 0.0
last byte: 192 192
final val to be sent back: 0.0
original sent bytes: b'\xf0\xf3\xf1\xf9\xf7\xf9\xf8\xf8\xf0\xc2'
c

last byte: 64 64
final val to be sent back: 0.0
original sent bytes: b'@@@@@@@@@@'
converted to array: array('B', [64, 64, 64, 64, 64, 64, 64, 64, 64, 64])
result by adding array: 0.0
last byte: 64 64
final val to be sent back: 0.0
original sent bytes: b'\xf0\xf3\xf2\xf0\xf3\xf9\xf7\xf2\xf9\xc4'
converted to array: array('B', [240, 243, 242, 240, 243, 249, 247, 242, 249, 196])
result by adding array: 320397294.0
last byte: 196 196
final val to be sent back: 32.0397294
original sent bytes: b'\xf0\xf9\xf5\xf5\xf4\xf3\xf5\xf9\xf4\xc9'
converted to array: array('B', [240, 249, 245, 245, 244, 243, 245, 249, 244, 201])
result by adding array: 955435949.0
last byte: 201 201
final val to be sent back: 95.5435949
original sent bytes: b'\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xc0'
converted to array: array('B', [240, 240, 240, 240, 240, 240, 240, 240, 240, 192])
result by adding array: 0.0
last byte: 192 192
final val to be sent back: 0.0
original sent bytes: b'\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\

In [6]:
split_records

[['00100001',
  {'RRC-TAPE-RECORD-ID': '13',
   'WB-LOC-COUNTY': 1,
   'WB-ABSTRACT': '',
   'WB-SURVEY': 'ELLIS',
   'WB-BLOCK-NUMBER': '',
   'WB-SECTION': '',
   'WB-ALT-SECTION': '',
   'WB-ALT-ABSTRACT': '',
   'WB-FEET-FROM-SUR-SECT-1': 0,
   'WB-DIREC-FROM-SUR-SECT-1': '',
   'WB-FEET-FROM-SUR-SECT-2': 0,
   'WB-DIREC-FROM-SUR-SECT-2': '',
   'WB-WGS84-LATITUDE': 32.0018943,
   'WB-WGS84-LONGITUDE': 96.0323202,
   'WB-PLANE-ZONE': 0,
   'WB-PLANE-COORDINATE-EAST': 0.0,
   'WB-PLANE-COORDINATE-NORTH': 0.0,
   'WB-VERIFICATION-FLAG': '0'}],
 ['00100008',
  {'RRC-TAPE-RECORD-ID': '13',
   'WB-LOC-COUNTY': 1,
   'WB-ABSTRACT': '',
   'WB-SURVEY': '',
   'WB-BLOCK-NUMBER': '',
   'WB-SECTION': '',
   'WB-ALT-SECTION': '',
   'WB-ALT-ABSTRACT': '',
   'WB-FEET-FROM-SUR-SECT-1': 0,
   'WB-DIREC-FROM-SUR-SECT-1': '',
   'WB-FEET-FROM-SUR-SECT-2': 0,
   'WB-DIREC-FROM-SUR-SECT-2': '',
   'WB-WGS84-LATITUDE': 31.9798802,
   'WB-WGS84-LONGITUDE': 96.0097968,
   'WB-PLANE-ZONE': None,
   'W

In [13]:
codecs.decode(b'0xD')

'0xD'